In [1]:
# ==============================
# 1. Install dependencies
# ==============================
!pip install -q transformers datasets accelerate peft bitsandbytes

In [2]:
# ==============================
# 2. Import libraries
# ==============================
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


c:\Demo\module05\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [3]:
# ==============================
# 3. Load model & tokenizer
# ==============================
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

c:\Demo\module05\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SanjeetKumar\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download.

In [4]:
# ==============================
# 4. Load a free sample dataset
# ==============================
# We'll use the "squad" dataset for QA fine-tuning as an example
dataset = load_dataset("squad", split="train[:1%]")  # small subset for Colab
val_dataset = load_dataset("squad", split="validation[:1%]")

# Preprocess: text -> input_ids, target -> labels
def preprocess(example):
    input_text = example["question"] + " context: " + example["context"]
    target_text = example["answers"]["text"][0] if len(example["answers"]["text"]) > 0 else ""
    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=32)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset.map(preprocess, batched=False)
tokenized_val = val_dataset.map(preprocess, batched=False)

c:\Demo\module05\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SanjeetKumar\.cache\huggingface\hub\datasets--squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better pe

In [5]:

# ==============================
# 5. Setup Training Arguments
# ==============================
training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_steps=50,
    save_strategy="epoch",

    eval_steps=100,
    save_total_limit=2,
    fp16=True,
    report_to="none",  # disable wandb
)



In [6]:
# ==============================
# 6. Trainer
# ==============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# ==============================
# 7. Start Training
# ==============================
trainer.train()

# ==============================
# 8. Save model & tokenizer
# ==============================
model.save_pretrained("./flan_t5_finetuned")
tokenizer.save_pretrained("./flan_t5_finetuned")



c:\Demo\module05\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,27.476800
100,12.975700
150,7.304100
200,5.667400


('./flan_t5_finetuned\\tokenizer_config.json',
 './flan_t5_finetuned\\special_tokens_map.json',
 './flan_t5_finetuned\\spiece.model',
 './flan_t5_finetuned\\added_tokens.json',
 './flan_t5_finetuned\\tokenizer.json')

In [7]:
# ==============================
# 9. Test inference
# ==============================
test_text = "Who wrote the book 'Pride and Prejudice'? context: Jane Austen wrote the novel 'Pride and Prejudice' in 1813."
inputs = tokenizer(test_text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print("Answer:", tokenizer.decode(outputs[0], skip_special_tokens=True))

Answer: edward w. edward
